In [1]:
import sys
import os
import shutil
import datetime
import re
import pandas as pd
from collections import Counter
#sys.path.append(os.path.abspath("/home/jose/Dropbox/Doktorarbeit/"))
#from reading_robot import load_data, tei2text, text2features, classify, process_labels, describe_data,feature_analysis, cull_data, st_tests, metadata2numbers, cluster, call_stylo, prepare_subcorpus
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copyfile
import numpy as np
import glob

# Read Data

In [2]:
files = glob.glob("./../results-new/*.csv")
df = pd.concat( [ pd.read_csv(file, index_col=0) for file in files])

In [3]:
df

,corpus,version,mfw,features,ngrams,distances,media
30,cuentos-espanoles,annotation,2000,w,2,dist.delta,100.000000
38,cuentos-espanoles,annotation,3000,w,2,dist.delta,100.000000
46,cuentos-espanoles,annotation,4000,w,2,dist.delta,100.000000
54,cuentos-espanoles,annotation,5000,w,2,dist.delta,100.000000
55,cuentos-espanoles,annotation,5000,w,3,dist.delta,100.000000
...,...,...,...,...,...,...,...
40,teatro-sgo,only-tokens,3000,w,4,dist.delta,11.472222
48,teatro-sgo,only-tokens,4000,w,4,dist.delta,11.472222
56,teatro-sgo,only-tokens,5000,w,4,dist.delta,11.472222
112,teatro-sgo,only-tokens,5000,w,4,dist.eder,11.472222


In [4]:
df.sort_values(by="mfw", inplace=True)

In [5]:
df.reset_index(inplace=True, drop=True)

In [6]:
df

,corpus,version,mfw,features,ngrams,distances,media
0,novelas-hispanoamericanas,combination,100,w,4,dist.minmax,56.607143
1,disco,combination,100,c,1,dist.minmax,35.441089
2,disco,combination,100,c,3,dist.eder,36.600046
3,disco,combination,100,c,4,dist.eder,36.613470
4,disco,combination,100,c,4,dist.minmax,37.338606
...,...,...,...,...,...,...,...
7035,prosa-SdO,fusion,10000,w,4,dist.minmax,70.000000
7036,cuentos-espanoles,fusion,10000,w,3,dist.eder,94.000000
7037,cuentos-espanoles,fusion,10000,w,2,dist.eder,94.000000
7038,prosa-SdO,annotation,10000,c,4,dist.eder,61.666667


In [7]:
df.to_csv("./../results/202209_all.tsv", sep="\t")

In [8]:
df = pd.read_csv("./../results/202209_all.tsv", index_col=0, sep="\t")

In [9]:
df

,corpus,version,mfw,features,ngrams,distances,media
0,novelas-hispanoamericanas,combination,100,w,4,dist.minmax,56.607143
1,disco,combination,100,c,1,dist.minmax,35.441089
2,disco,combination,100,c,3,dist.eder,36.600046
3,disco,combination,100,c,4,dist.eder,36.613470
4,disco,combination,100,c,4,dist.minmax,37.338606
...,...,...,...,...,...,...,...
7035,prosa-SdO,fusion,10000,w,4,dist.minmax,70.000000
7036,cuentos-espanoles,fusion,10000,w,3,dist.eder,94.000000
7037,cuentos-espanoles,fusion,10000,w,2,dist.eder,94.000000
7038,prosa-SdO,annotation,10000,c,4,dist.eder,61.666667


In [10]:
df.corpus.unique()

array(['novelas-hispanoamericanas', 'disco', 'prosa-SdO',
       'Poesia_SiglodeOro', 'cuentos-espanoles', 'teatro-sgo',
       'novelas-espanolas'], dtype=object)

# Filter Data

In [11]:
filtered_df = df.copy()

In [12]:
df

,corpus,version,mfw,features,ngrams,distances,media
0,novelas-hispanoamericanas,combination,100,w,4,dist.minmax,56.607143
1,disco,combination,100,c,1,dist.minmax,35.441089
2,disco,combination,100,c,3,dist.eder,36.600046
3,disco,combination,100,c,4,dist.eder,36.613470
4,disco,combination,100,c,4,dist.minmax,37.338606
...,...,...,...,...,...,...,...
7035,prosa-SdO,fusion,10000,w,4,dist.minmax,70.000000
7036,cuentos-espanoles,fusion,10000,w,3,dist.eder,94.000000
7037,cuentos-espanoles,fusion,10000,w,2,dist.eder,94.000000
7038,prosa-SdO,annotation,10000,c,4,dist.eder,61.666667


In [13]:
df.loc[(df["corpus"] == "prosa-SdO") & (df["version"] == "only-tokens") & (df["features"] == "c") & (df["distances"] == "dist.delta") & (df["ngrams"] == 3) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media
244,prosa-SdO,only-tokens,100,c,3,dist.delta,64.166667
1644,prosa-SdO,only-tokens,500,c,3,dist.delta,64.166667
2354,prosa-SdO,only-tokens,1000,c,3,dist.delta,70.000000
2838,prosa-SdO,only-tokens,2000,c,3,dist.delta,73.333333
3692,prosa-SdO,only-tokens,3000,c,3,dist.delta,67.500000
4932,prosa-SdO,only-tokens,4000,c,3,dist.delta,67.500000
5470,prosa-SdO,only-tokens,5000,c,3,dist.delta,64.166667
6574,prosa-SdO,only-tokens,7000,c,3,dist.delta,64.166667
6706,prosa-SdO,only-tokens,10000,c,3,dist.delta,64.166667


In [14]:
df.loc[(df["corpus"] == "prosa-SdO") & (df["version"] == "annotation") & (df["features"] == "w") & (df["distances"] == "dist.eder") & (df["ngrams"] == 1) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media
75,prosa-SdO,annotation,100,w,1,dist.eder,61.666667
1059,prosa-SdO,annotation,500,w,1,dist.eder,61.666667
2288,prosa-SdO,annotation,1000,w,1,dist.eder,61.666667
2899,prosa-SdO,annotation,2000,w,1,dist.eder,61.666667
3695,prosa-SdO,annotation,3000,w,1,dist.eder,61.666667
4533,prosa-SdO,annotation,4000,w,1,dist.eder,61.666667
5962,prosa-SdO,annotation,5000,w,1,dist.eder,61.666667
6330,prosa-SdO,annotation,7000,w,1,dist.eder,61.666667
7028,prosa-SdO,annotation,10000,w,1,dist.eder,61.666667


In [15]:
df.loc[(df["corpus"] == "teatro-sgo") & (df["version"] == "annotation") & (df["features"] == "w") & (df["distances"] == "dist.delta") & (df["ngrams"] == 1) & (df["mfw"] >= 100)]


,corpus,version,mfw,features,ngrams,distances,media
860,teatro-sgo,annotation,100,w,1,dist.delta,56.277778
1577,teatro-sgo,annotation,500,w,1,dist.delta,63.111111
2376,teatro-sgo,annotation,1000,w,1,dist.delta,63.111111
2709,teatro-sgo,annotation,2000,w,1,dist.delta,63.111111
4374,teatro-sgo,annotation,3000,w,1,dist.delta,63.111111
4893,teatro-sgo,annotation,4000,w,1,dist.delta,63.111111
5518,teatro-sgo,annotation,5000,w,1,dist.delta,63.111111


In [16]:
df.loc[(df["corpus"] == "teatro-sgo") & (df["version"] == "combination") & (df["features"] == "w") & (df["distances"] == "dist.delta") & (df["ngrams"] == 2) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media
778,teatro-sgo,combination,100,w,2,dist.delta,58.083333
1051,teatro-sgo,combination,500,w,2,dist.delta,62.944444
2309,teatro-sgo,combination,1000,w,2,dist.delta,62.833333
3140,teatro-sgo,combination,2000,w,2,dist.delta,69.194444
4439,teatro-sgo,combination,3000,w,2,dist.delta,65.833333
4643,teatro-sgo,combination,4000,w,2,dist.delta,69.944444
5534,teatro-sgo,combination,5000,w,2,dist.delta,64.944444


In [17]:
df.loc[(df["corpus"] == "Poesia-SdO") & (df["version"] == "fusion") & (df["features"] == "w") & (df["distances"] == "dist.delta") & (df["ngrams"] == 2) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media


In [18]:

df.loc[(df["corpus"] == "novelas-espanolas") & (df["version"] == "fusion") & (df["features"] == "w") & (df["distances"] == "dist.delta") & (df["ngrams"] == 2) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media
408,novelas-espanolas,fusion,100,w,2,dist.delta,82.686480
1073,novelas-espanolas,fusion,500,w,2,dist.delta,88.822844
1955,novelas-espanolas,fusion,1000,w,2,dist.delta,87.144522
3365,novelas-espanolas,fusion,2000,w,2,dist.delta,87.068765
3703,novelas-espanolas,fusion,3000,w,2,dist.delta,87.068765
5196,novelas-espanolas,fusion,4000,w,2,dist.delta,87.762238
5457,novelas-espanolas,fusion,5000,w,2,dist.delta,86.928904


In [19]:
df.loc[(df["corpus"] == "prosa-SdO") & (df["version"] == "fusion") & (df["features"] == "w") & (df["distances"] == "dist.eder") & (df["ngrams"] == 2) & (df["mfw"] >= 100)]

,corpus,version,mfw,features,ngrams,distances,media
308,prosa-SdO,fusion,100,w,2,dist.eder,65.000000
1477,prosa-SdO,fusion,500,w,2,dist.eder,68.333333
2413,prosa-SdO,fusion,1000,w,2,dist.eder,68.333333
3074,prosa-SdO,fusion,2000,w,2,dist.eder,68.333333
3779,prosa-SdO,fusion,3000,w,2,dist.eder,68.333333
4793,prosa-SdO,fusion,4000,w,2,dist.eder,68.333333
5926,prosa-SdO,fusion,5000,w,2,dist.eder,68.333333
6522,prosa-SdO,fusion,7000,w,2,dist.eder,68.333333
6968,prosa-SdO,fusion,10000,w,2,dist.eder,68.333333


In [20]:
len(df.loc[(df["corpus"] == "prosa-SdO") & (df["version"] == "annotation") & (df["features"] == "w") & (df["distances"] == "dist.eder") & (df["ngrams"] == 1) & (df["mfw"] >= 100)]["media"].unique())


1

In [21]:
for corpus in df["corpus"].unique():
    for version in df["version"].unique():
        for ngram in df["ngrams"].unique():
            for features in df["features"].unique():
                for mfw in df["mfw"].unique():
                    media_with_mfw = df.loc[(df["corpus"] == corpus) & (df["version"] == version) & (df["features"] == features) & (df["distances"] == "dist.delta") & (df["ngrams"] == ngram) & (df["mfw"] == mfw)]

                    if len(df.loc[(df["corpus"] == corpus) & (df["version"] == version) & (df["features"] == features) & (df["distances"] == "dist.delta") & (df["ngrams"] == ngram) & (df["mfw"] >= mfw)]["media"].unique()) == 1:
                        try:
                            filtered_df.drop(df.loc[(df["corpus"] == corpus) & (df["version"] == version) & (df["features"] == features) & (df["ngrams"] == ngram) & (df["mfw"] > mfw)].index, inplace=True)
                        except:
                            pass

In [22]:
filtered_df

,corpus,version,mfw,features,ngrams,distances,media
0,novelas-hispanoamericanas,combination,100,w,4,dist.minmax,56.607143
1,disco,combination,100,c,1,dist.minmax,35.441089
2,disco,combination,100,c,3,dist.eder,36.600046
3,disco,combination,100,c,4,dist.eder,36.613470
4,disco,combination,100,c,4,dist.minmax,37.338606
...,...,...,...,...,...,...,...
6265,disco,fusion,5000,w,2,dist.delta,42.506842
6267,disco,only-tokens,5000,w,1,dist.wurzburg,75.415982
6268,Poesia_SiglodeOro,only-tokens,5000,w,1,dist.eder,87.285741
6269,Poesia_SiglodeOro,only-tokens,5000,w,1,dist.wurzburg,87.234585


In [24]:
filtered_df.to_csv("./../results/202210_all_filtered.tsv", sep="\t")